In [ ]:
from flask import Flask, request, jsonify
from sentence_transformers import SentenceTransformer
import logging

logging.basicConfig(level=logging.DEBUG, 
                    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s')

app = Flask(__name__)
logger = logging.getLogger(__name__)

# 模型加载（带错误处理）
try:
    model = SentenceTransformer(r"E:\RAG\embedding\qwen3-embedding")
    logger.info("✅ 模型加载成功")
except Exception as e:
    logger.error(f"❌ 模型加载失败: {str(e)}")
    # 实际部署时应退出或通知运维
    # raise SystemExit("模型加载失败，服务终止")

@app.route('/embed', methods=['POST'])
def get_embedding():
    try:
        # 请求数据验证
        data = request.get_json()
        if not data:
            return jsonify({"error": "请求体为空"}), 400
        
        # 支持单文本(text)或批量(texts)处理
        texts = data.get('texts') or [data.get('text')]
        if not texts or any(not t for t in texts):
            return jsonify({"error": "无效的文本输入"}), 400
        
        # 生成嵌入向量
        embeddings = model.encode(texts)
        result = {
            "embeddings": [emb.tolist() for emb in embeddings],
            "count": len(embeddings)
        }
        return jsonify(result)
    
    except Exception as e:
        logger.exception("处理请求时出错")
        return jsonify({"error": "内部服务器错误"}), 500

@app.route('/health', methods=['GET'])
def health_check():
    return jsonify({"status": "healthy", "model": model.__class__.__name__})

if __name__ == '__main__':
    app.run(host='0.0.0.0', port=8888)


2025-08-30 02:28:11,672 - sentence_transformers.SentenceTransformer - INFO - Use pytorch device_name: cpu
2025-08-30 02:28:11,672 - sentence_transformers.SentenceTransformer - INFO - Load pretrained SentenceTransformer: E:\RAG\embedding\qwen3-embedding
2025-08-30 02:28:13,686 - sentence_transformers.SentenceTransformer - INFO - 1 prompt is loaded, with the key: query
2025-08-30 02:28:13,686 - __main__ - INFO - ✅ 模型加载成功


 * Serving Flask app '__main__'
 * Debug mode: off


2025-08-30 02:28:13,704 - werkzeug - INFO - WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:8888
 * Running on http://192.168.10.193:8888
2025-08-30 02:28:13,704 - werkzeug - INFO - Press CTRL+C to quit


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

2025-08-30 02:28:22,176 - werkzeug - INFO - 127.0.0.1 - - [30/Aug/2025 02:28:22] "POST /embed HTTP/1.1" 200 -
